In [4]:
from rdflib import Graph
import httpx
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore
from pyfuseki import FusekiUpdate

In [2]:
#Work
uri = 'http://id.loc.gov/resources/works/748300.bibframe.rdf'
g = Graph()
g.parse(uri)
g.serialize('work.ttl')

<Graph identifier=Nfa6545d22eb44b79ba5034d67282a9fb (<class 'rdflib.graph.Graph'>)>

In [3]:
#Instancia
uri = 'http://id.loc.gov/resources/instances/748300.bibframe.rdf'
g = Graph()
g.parse(uri)
g.serialize('instance.ttl')

<Graph identifier=N1a3c9c4eefcc406ca09204980baffb0b (<class 'rdflib.graph.Graph'>)>

In [ ]:
#Item
uri = 'http://id.loc.gov/resources/items/748300-050-10.bibframe.rdf'
g = Graph()
g.parse(uri)
g.serialize('item.nt')

In [10]:
#Subject
#uri = 'http://id.loc.gov/authorities/subjects/sh85084414.rdf'
uri = "http://id.loc.gov/authorities/subjects/sh85084414.madsrdf.rdf"
g = Graph()
g.parse(uri)
g.serialize('Methodology.nt')

<Graph identifier=Nc26ecb5a7ba8461b9a2e8d7a2c313fe6 (<class 'rdflib.graph.Graph'>)>

In [60]:
#Subject BK
token = "sh85084414"
uri = f'http://localhost:3030/thesaurus?graph=https://bibliokeia.com/authorities/subjects/{token}'
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('Metodologia.nt')

<Graph identifier=Nbb962704343b4c63b909022dcf2a70ca (<class 'rdflib.graph.Graph'>)>

Names

In [29]:
uri = 'http://id.loc.gov/authorities/names/n80032184.madsrdf.rdf'
g = Graph()
g.parse(uri)
g.serialize('PopperLOC.nt')

<Graph identifier=Nbb1cff5129c44881acd8f5fc6326e769 (<class 'rdflib.graph.Graph'>)>

NAMES BK

In [58]:
uri = "http://localhost:3030/authority?graph=https://bibliokeia.com/authorities/names/n80002329"
r = httpx.get(uri)
r



<Response [200 OK]>

In [59]:
uri = "http://localhost:3030/authority?graph=https://bibliokeia.com/authorities/names/n80002329"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('Machado.nt')

<Graph identifier=N7be6e5a3603c444e86493d62d3ec549f (<class 'rdflib.graph.Graph'>)>

In [2]:
from pyfuseki import FusekiQuery

In [3]:
fuseki_query = FusekiQuery('http://localhost:3030', 'thesaurus')

GRAPH EXISTS

In [4]:
query = """PREFIX bk: <https://bibliokeia.com/authorities/subjects/>
  ASK WHERE { GRAPH bk:sh85066157 { ?s ?p ?o } }"""

query_result = fuseki_query.run_sparql(query)
query_result

In [4]:
query = """
    PREFIX madsrdf:  <http://www.loc.gov/mads/rdf/v1#> 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX bk: <https://bibliokeia.com/authorities/subjects/>
SELECT (COUNT(*) AS ?count)
WHERE { GRAPH  bk:sh85084414
      {?s ?p ?o . }
    }
"""

query_result = fuseki_query.run_sparql(query)
query_result

In [28]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore

In [38]:
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/authority/update')
query_endpoint = 'http://localhost:3030/authority/query'
update_endpoint = 'http://localhost:3030/authority/update'
store.open((query_endpoint, update_endpoint))

In [51]:
uri = "http://id.loc.gov/authorities/names/n80032184"

up = """PREFIX bflc: <http://id.loc.gov/ontologies/bflc/>
PREFIX Namesbk: <https://bibliokeia.com/authorities/names/> 
                INSERT DATA
                { GRAPH  <https://bibliokeia.com/authorities/names/n80032184>{ 
                    <https://bibliokeia.com/authorities/names/n80032184> 
                     bflc:contributorTo 
                     <https://bibliokeia.com/bibframe/work/TESTE2> } }"""
store.update(up)

In [54]:
uri = "http://localhost:3030/authority?graph=https://bibliokeia.com/authorities/names/n80032184"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('PopperBK.nt')

<Graph identifier=N9e84d1e4499246bb96162da97a89a6f2 (<class 'rdflib.graph.Graph'>)>

In [17]:
def GraphExist(token):
    store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/thesaurus/update')
    query_endpoint = 'http://localhost:3030/thesaurus/query'
    update_endpoint = 'http://localhost:3030/thesaurus/update'
    store.open((query_endpoint, update_endpoint))

    query = "PREFIX bk: <https://bibliokeia.com/authorities/subjects/>\n \
                ASK WHERE { GRAPH bk:" + token +" { ?s ?p ?o } }"
    
    response = store.query(query)

    return response.askAnswer

In [19]:

def UpdateThesarus(graph, metadata, token):

    store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/thesaurus/update')
    query_endpoint = 'http://localhost:3030/thesaurus/query'
    update_endpoint = 'http://localhost:3030/thesaurus/update'
    store.open((query_endpoint, update_endpoint))

    prefix = "PREFIX madsrdf:  <http://www.loc.gov/mads/rdf/v1#>\n \
        PREFIX bk: <https://bibliokeia.com/authorities/subjects/>\n \
        PREFIX lc: <http://id.loc.gov/authorities/subjects/>\n"

    q1 = prefix + "WITH bk:" + graph + "\n \
        DELETE {?s madsrdf:" + metadata + " lc:" + token + "}\n \
        INSERT { ?s madsrdf:" + metadata + " bk:" + token + "}\n \
        WHERE { ?s madsrdf:" + metadata + " lc:" + token + "}\n"
    store.update(q1)

    q2 = prefix + "WITH bk:" + graph + "\n \
            DELETE { lc:" + metadata + " ?p ?o }\n \
                INSERT { bk:" + metadata + " ?p ?o }\n \
                    WHERE { lc:" + metadata + "?p ?o }"
    store.update(q2)

    q3 = prefix + "WITH bk:" + graph + "\n \
            DELETE { lc:" + metadata + " madsrdf:isMemberOfMADSCollection ?o }\n \
                INSERT { bk:" + metadata + " madsrdf:isMemberOfMADSCollection bk:collection_BKSH_General }\n \
                    WHERE { lc:" + metadata + " madsrdf:isMemberOfMADSCollection ?o }"
    store.update(q3)

False

In [27]:
prefix = "PREFIX madsrdf:  <http://www.loc.gov/mads/rdf/v1#>\n \
        PREFIX bk: <https://bibliokeia.com/authorities/subjects/>\n \
        PREFIX lc: <http://id.loc.gov/authorities/subjects/>\n"

In [30]:
graph = "sh85084414"
token = "sh85113021"
q2 = prefix + "WITH bk:" + graph + "\n \
            DELETE { bk:sh85113021 ?p lc:collection_LCSH_General }\n \
                INSERT { bk:sh85113021 ?p bk:collection_BKSH_General}\n \
                    WHERE { bk:sh85113021 ?p lc:collection_LCSH_General  }"
store.update(q2)


In [ ]:
q1 = prefix + "WITH bk:" + graph + "\n \
        DELETE {?s madsrdf:" + metadata + " lc:" + token + "}\n \
        INSERT { ?s madsrdf:" + metadata + " bk:" + token + "}\n \
        WHERE { ?s madsrdf:" + metadata + " lc:" + token + "}\n"

In [ ]:
q3 = prefix + "WITH bk:sh85084414 \n \
            DELETE { bk:sh85113021 madsrdf:isMemberOfMADSCollection ?o }\n \
                INSERT { bk:sh85113021 madsrdf:isMemberOfMADSCollection bk:collection_BKSH_General }\n \
                    WHERE { bk:sh85113021 madsrdf:isMemberOfMADSCollection ?o }"
store.update(q3)

UPDATE WORK

In [5]:
fuseki_update = FusekiUpdate('http://localhost:3030', 'acervo')

In [10]:
up = """PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>
        INSERT DATA
            { GRAPH  <http://bibliokeia.com/bibframe/work/bk-23-10> { 
                    <https://bibliokeia.com/bibframe/work/bk-23-10>  
                    bf:hasInstance
                    <http://bibliokeia.com/bibframe/instance/bk-23-10> } }"""

In [11]:
fuseki_update.run_sparql(up)